In [2]:
import pprint
import os
from rdflib import Namespace
from cmmn3 import parseModel, convertModel, convertLog, convertCtx, reasonAll, showCase, aggregateErrors, showErrors

n3Path = "cmmn3/n3/"
xmlNs = {"cmmn": "http://www.omg.org/spec/CMMN/20151109/MODEL"}
modelNs = Namespace("http://ontario.org/qbp/cap#")

xmlPath = "/Users/wvw/Dropbox/research/projects/Montfort PM/cmmn/cap-all.cmmn"
modelPath = "ttl/cap/model.ttl"
csvFolder = "/Users/wvw/git/pm/mft/data/subsets/"
evtPath = os.path.join(csvFolder, "cap-events-labreq_map.csv")
# statCtxPath = os.path.join(csvFolder, "cap-status-admit.csv")
dynCtxPath = os.path.join(csvFolder, "cap-clinical_data.csv")
obsPath = "ttl/cap/obs/obs_all.ttl"
ctxPath = "ttl/cap/ctx"
imgPath = "js/diagram.png"

# Convert TTL

## Model

In [2]:
itemObjs = parseModel(xmlPath, xmlNs)
# pprint.pprint(itemObjs)
convertModel(itemObjs, modelNs, modelPath)

## Log

In [3]:
convertLog(evtPath, modelNs, destPath=obsPath)

## Ctx

In [5]:
convertCtx(evtPath, dynCtxPath, modelNs, destPath=ctxPath)

case #0 / 3400


/Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/convert.py:47: DtypeWarning: Columns (34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  value = row['value']


# Check compliance

## Reason

In [ ]:
out = reasonAll(modelPath, obsPath, n3Path, printerr=False)
errors, finals = out
print("errors:", errors.columns)
print("finals:", finals.columns)

In [25]:
from util import run

out = run(['eye', 
        "ttl/cap/ctx/static.n3",
        "ttl/cap/ctx/dynamic.n3",
        "cmmn3/n3/calc.n3",
        '--query', "cmmn3/n3/test.n3",
        '--nope'], printerr=True, get_time=False)
print(out)

error: eye --quiet ttl/cap/ctx/static.n3 ttl/cap/ctx/dynamic.n3 cmmn3/n3/calc.n3 --query cmmn3/n3/test.n3 --nope
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 25 [msec cputime] 27 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/ttl/cap/ctx/static.n3 SC=2
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/ttl/cap/ctx/dynamic.n3 SC=4
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/n3/calc.n3 SC=7
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/n3/test.n3 SC=1
networking 16 [msec cputime] 19 [msec walltime]
reasoning 5 [msec cputime] 4 [msec walltime]
2026-03-01T15:49:54.723Z in=14 out=5 ent=5 step=5 brake=2 inf=375898 sec=0.046 inf/sec=8171696


# Processed by EYE v11.23.2 (2025-12-09)
# eye --quiet ttl/cap/ctx/static.n3 ttl/cap/ctx/dynamic.n3 cmmn3/n3/calc.n3 --query cmmn3/n3/test.n3 --nope

@prefix ns3: <http://rdf.org/test#>.

(<http://ontario.org/qbp/cap#case_13> 0) ns3:test (() 0).
(<http://ontario.org/qbp/cap#case_88> 0) ns3:test (() 0).
(<http://ontario.org/

## Show

In [ ]:
itemObjs = parseModel(xmlPath, xmlNs)

In [ ]:
showCase(out, '100057', itemObjs, xmlPath, imgPath)

In [ ]:
agg_errors = aggregateErrors(out)
agg_errors

In [ ]:
errors[errors['item']=='oxygen saturation']

In [ ]:
# TODO inactiveToCompleted: show on outer stage

showErrors(agg_errors, itemObjs, xmlPath, imgPath)

In [ ]:
from IPython.display import Image, display

display(Image(filename=imgPath))